# Checks

In [1]:
import torch

# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    print("GPU is available!")
    print(f"GPU device name: {torch.cuda.get_device_name(0)}")
else:
    print("GPU is not available.")


GPU is available!
GPU device name: Tesla V100-PCIE-32GB


In [2]:
import os

def is_slurm_job():
    # Check if common SLURM environment variables are set
    return 'SLURM_JOB_ID' in os.environ or 'SLURM_TASKS_PER_NODE' in os.environ

# Check if the Jupyter notebook is running under a SLURM job
if is_slurm_job():
    print("Running on a SLURM-managed cluster.")
    print(f"SLURM_JOB_ID: {os.environ.get('SLURM_JOB_ID')}")
    print(f"SLURM_NODELIST: {os.environ.get('SLURM_NODELIST')}")
else:
    print("Running locally (not on a SLURM-managed cluster).")

Running on a SLURM-managed cluster.
SLURM_JOB_ID: 912556
SLURM_NODELIST: atl1-1-01-002-1-0


# Imports

In [7]:
%load_ext autoreload

In [6]:
import sys
sys.path.append('/home/hice1/mbibars3/scratch/vlm-debiasing/VLM-Debiasing-Project/scripts')

In [8]:
%autoreload 2
import extract_audio_features

In [38]:
from transformers import AutoFeatureExtractor, ASTForAudioClassification
import pandas as pd

In [5]:
# Load the labels CSV
csv_path = "/home/hice1/mbibars3/scratch/vlm-debiasing/data/e-daic/original/labels/detailed_lables.csv"
labels_df = pd.read_csv(csv_path)
labels_df.head()

,Participant,PHQ8_1_NoInterest,PHQ8_2_Depressed,PHQ8_3_Sleep,PHQ8_4_Tired,PHQ8_5_Appetite,PHQ8_6_Failure,PHQ8_7_Concentration,PHQ8_8_Psychomotor,Depression_severity,...,PCL-C_14_Irritability,PCL-C_15_Concentration,PCL-C_16_HyperAlert,PCL-C_17_Jumpy,PTSD_severity,gender,age,Depression_label,PTSD_label,split
0,300,0,0,1,0,1,0,0,0,2,...,2,2,2,1,25.0,male,33,0,0,dev
1,301,0,0,1,1,1,0,0,0,3,...,1,1,1,1,17.0,male,39,0,0,dev
2,302,1,1,0,1,0,1,0,0,4,...,1,1,1,1,28.0,male,25,0,0,train
3,303,0,0,0,0,0,0,0,0,0,...,1,1,1,1,17.0,female,41,0,0,train
4,304,0,1,1,2,2,0,0,0,6,...,1,2,2,1,20.0,female,22,0,0,train


In [9]:
main_dir = "/home/hice1/mbibars3/scratch/vlm-debiasing/data/e-daic/untarred"
labels_csv = labels_df
cols = ["gender", "split", "PTSD_label","age","PTSD_severity"]

In [34]:
wav_paths_df = extract_audio_features.list_audio_files_paths(main_dir, labels_csv, cols)

100%|██████████| 275/275 [00:00<00:00, 924.63it/s]


In [35]:
wav_paths_df.head()

,gender,split,PTSD_label,age,PTSD_severity,file_path
0,female,train,0,45,22.0,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
1,male,test,0,69,23.0,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
2,male,train,0,25,19.0,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
3,female,train,1,58,67.0,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
4,male,dev,0,33,39.0,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...


In [36]:
path_to_save = "/home/hice1/mbibars3/scratch/vlm-debiasing/data/e-daic"
file_name = "labels_audio_wavs.csv"
extract_audio_features.save_csv(wav_paths_df, path_to_save, file_name)

# Data Reading

In [50]:
path = "/home/hice1/mbibars3/scratch/vlm-debiasing/data/e-daic/labels_audio_wavs.csv"
df = pd.read_csv(path)
wav_paths_csv = df["file_path"]

In [39]:
target_path = "/home/hice1/mbibars3/scratch/vlm-debiasing/data/e-daic/untarred"
# Load model and processor
model_name = "MIT/ast-finetuned-audioset-10-10-0.4593"
processor = AutoFeatureExtractor.from_pretrained(model_name)
model = ASTForAudioClassification.from_pretrained(model_name, output_hidden_states=True)

In [43]:
# output feature size: (1214, 768)
extract_audio_features.extract_features_AST(wav_paths_csv, target_path, processor, model)

100%|██████████| 275/275 [03:05<00:00,  1.48it/s]


In [51]:
# Add the new column with modified file paths
df["npy_file_path"] = df["file_path"].str.replace(".wav", "_ast9.npy", regex=False)
df.head()

,gender,split,PTSD_label,age,PTSD_severity,file_path,npy_file_path
0,female,train,0,45,22.0,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
1,male,test,0,69,23.0,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
2,male,train,0,25,19.0,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
3,female,train,1,58,67.0,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
4,male,dev,0,33,39.0,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...


In [52]:
df["file_path"][0]

'/home/hice1/mbibars3/scratch/vlm-debiasing/data/e-daic/untarred/449_P/449_AUDIO.wav'

In [54]:
import numpy as np
np.load(df["npy_file_path"][0]).shape

(1214, 768)